In [7]:
import numpy as np
import os
from pydrake.common import temp_directory, FindResourceOrThrow
from pydrake.geometry import (
    MeshcatVisualizer, MeshcatVisualizerParams, 
    Role, StartMeshcat
)

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer

from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths

In [8]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [15]:
hand_url = FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"
)
arm_url = '../robot_models/planar_2R.sdf'

full_2r_directives = '''
directives: 
- add_model: 
    name: 2R_planar_robot
    file: file:///home/dema/PycharmProjects/robot_manipulator/robot_models/Planar-v2.sdf
    default_joint_positions: 
        q1: [0]
        q2: [0]


- add_model: 
    name: end_effector
    file: file:///home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld: 
    parent: 2R_planar_robot::l2
    child: end_effector::body_frame
    X_PC: 
        translation: [0, 0.5, 0]
        rotation: !Rpy {deg: [0, 0, 1.6]}
'''


In [19]:

builder = DiagramBuilder()
time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)

parser = Parser(plant)
parser.AddModelsFromString(full_2r_directives, '.dmd.yaml')
plant.Finalize()

BODY_NAMES = ['l1', 'l2', 'body']
for body_name in BODY_NAMES:  
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)
       
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
    
diagram.ForcedPublish(diagram_context)


In [20]:

simulator = Simulator(diagram, diagram_context)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)
visualizer.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(5.0)
visualizer.StopRecording()
visualizer.PublishRecording()


RuntimeError: Actuation input port for model instance 2R_planar_robot must be connected.